<h1><center>Équipe F</center></h1>
<h1>M1-ISDD</h1>
AUCLAIR LUCAS<br>
COUSTILLAS LAURÉDANE<br>
ZOHDY-CAUVIN SALMA<br>
<br>

# Régression linéaire et ACP avec statsmodels et scikit-learn

- Ce projet a été réalisé dans le cadre de l'UE Python2 du Master de Bioinformatique de l'Université Paris Cité.
- Le but de celui-ci est de réaliser une régression linéaire et une analyse ACP avec les bibliothèques statsmodels et scikit-learn ainsi que de comparer leurs utilisations.
- Le projet doit être un tutoriel, ce faisant, une vidéo tutoriel complémentaire a été réalisée en parallèle.
- Le présent Notebook existe pour réaliser celui-ci et présenter les données le plus clairement possible.

## La bibliothèque scikit-learn :

- Création : développée en 2007 par David Cournapeau dans le cadre du programme *Google Summer of Code*.
- La première version publique (v0.1 beta) est sortie en Janvier 2010 avec l'aide de l'INRIA (Institut National de Recherche en Informatique et en Automatique).
- Dernière version : 1.5.2 (11 Septembre, 2024).
- Pour installer dans un environnement conda :
```bash
$ conda install scikit-learn
```
Pour la version spécifique :
```bash
$ conda install scikit-learn=1.5.2
```

## La bibliothèque statsmodels  :

- Création : Par Skipper Seabold et Josef Perktold en 2009 dans le cadre du programme *Google Summer of Code*.
- Dernière version : 0.14.4 (Octobre 2024)
- Pour installer dans un environnement conda :
```bash
$ conda install statsmodels
```
Pour la version spécifique :
```bash
$ conda install statsmodels=0.14.4
```

## Points en commun entre les deux bibliothèques :

- Ce sont des outils importants pour l'analyse de données.
- Elles sont utilisées pour des modèles de statistiques (tests, régression linéaire) et de machine learning (notamment Scikit-learn).
- Elles sont compatibles avec d'autres bibliothèques usuelles en Python comme Numpy, Pandas et Matplotlib.
- Elles peuvent faire des ACP (Analyse en Composantes Principales, PCA en anglais).

## Importation des bibliothèques :

In [ ]:
# Pour la gestion du jeu de données.
import pandas as pd
# Pour les représentations graphiques.
import matplotlib.pyplot as plt
# Pour l'analyse statistique.
# .api est l'importation classique.
import statsmodels.api as sm
import sklearn as sk
# sous-bibliothèque essentielle.
import sklearn.linear_model

- Fixer la racine utilisée pour les nombres aléatoires afin d'éviter certaines variations mineures qui ont lieu lorsque le code est exécuté plusieurs fois ou lorsqu'il est exécuté sur différentes machines.
- Les méthodes utilisées pour l'analyse statistique sont déterministes et ne devraient pas varier normalement (ou très peu). Néanmoins, parfois, certaines différences ont pu être constatées, ceci permet donc d'améliorer la reproductibilité des données générées par le code et les analyses dans le notebook.

In [ ]:
# Bibliothèque importée pour définir la racine.
import numpy as np
# Définition de la racine.
np.random.seed(2024)

- Jupyter notebook magic command pour afficher ce que le code print.

In [ ]:
%matplotlib inline

## Importation des données **penguins_raw.csv** dans un Dataframe pour l'analyse statistique :

- Le fichier penguins_raw.csv contient des données écologiques collectées sur des manchots de trois espèces différentes : Adélie, Chinstrap, et Gentoo, originaires de l'archipel des îles Palmer en Antarctique.\
Ce jeu de données est une ressource populaire pour l'apprentissage des analyses statistiques et la visualisation de données. Il s'agit d'une alternative moderne et accessible au jeu de données historique "Iris" et illustre des concepts de biologie, d'écologie et d'analyse de données.
- Les données brutes ont été prises à la place des données simplifiées afin de réaliser une analyse de données et des comparaisons pertinentes (données simplifiées : penguins.csv)\
La colonne **Individual ID** est prise comme colonne des indices pour le Dataframe du jeu de données comme tous les IDs sont différents.

In [ ]:
df = pd.read_csv("penguins_raw.csv", index_col="Individual ID")

- Taille du Dataframe sous la forme (**lignes**, **colonnes**) :

In [ ]:
df.shape

- Vérifier si des valeurs Null sont présentes dans chaque colonne du jeu de données (*i.e* NaN or NA) :

In [ ]:
df.isnull().any()

- Comptabiliser les valeurs Null pour chaque colonne :

In [ ]:
df.isnull().sum()

- Regarder globalement à quoi ressemble le jeu de données : *"head"* pour les premières lignes du jeu de données et *"tail"* pour les dernières lignes de celui-ci.

In [ ]:
df.head()

In [ ]:
df.tail()

- On supprime la colonne **Comments** car elle ne servira pas à l'analyse. En effet, c'est celle qui contient le plus de valeurs Null, elle n'est donc pas pertinente pour notre analyse de données.
- Par ailleurs, les commentaires en question ont l'air de plus se référer à des critiques des observations, leur absence n'est donc pas un défaut mais plutôt une preuve que l'échantillon devrait être correct.
- De plus, cela évite d'enlever 290 lignes de notre jeu de données et permet donc d'éviter les biais induits par la diminution de la taille de celui-ci.\
**N.B.** : *axis = 0* pour les lignes et *axis = 1* pour les colonnes.

In [ ]:
df = df.drop('Comments', axis=1)

- On supprime toutes les lignes comportant des valeurs Null restantes (comme indiqué dans les consignes), la modification est faite **inplace/sur place** car nous n'avons pas besoin d'un sous-ensemble du jeu de données avec des valeurs Null.
- En ayant enlevé la colonne **Comments** au préalable, cela permet d'enlever moins de 30 lignes au total, là où sinon notre jeu de données à analyser serait complètement vide ou presque, ne l'ayant pas fait.
- **N.B. : Il est important de minimiser la diminution du nombre de variables dans le jeu de données pour une analyse pertinente**, ce faisant, il faut savoir quelles variables nous intéressent et quelle quantité de ces variables nous est nécessaire dans notre analyse. Ici, en enlevant toutes les valeurs Null après avoir enlevé la colonne **Comments**, on ne perd qu'au maximum 14 variables. Ce faisant, on considère qu'on peut garder les autres colonnes ou éviter d'utiliser d'autres méthodes plus complexes pour gérer un jeu de données contenant de nombreuses valeurs Null.

In [ ]:
df.dropna(inplace=True)

- On vérifie que les modifications faites sont correctes et correspondent à nos attentes : 

In [ ]:
df.head()

- On observe que tout s'est déroulé comme souhaité préalablement : 

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

## Principes de la régression linéaire :

### Définitions :

- En mathématiques et notamment en statistiques, une régression est un ensemble de méthodes différentes permettant d'approcher une variable à partir d'autres qui lui sont corrélées, le tout en créant un modèle mathématique qui approche le plus possible la variable étudiée et celles qui lui sont corrélées, ce qui permet à terme d'étudier les relations existantes (ou non) entre ces variables.<br><br>
- Par extension, un modèle de régression linéaire est un modèle qui tend à établir une relation linéaire entre une variable, dite *expliquée*, et une ou plusieurs variables, dites *explicatives*, souvent dans le but de prédire la variable *expliquée* à partir des variables *explicatives*.<br><br>
- **N.B.** : La régression linéaire repose sur l'algèbre linéaire et donc par extension par exemple la diagonalisation de matrices de covariance. Cela n'est absolument pas abordé ici comme le but est de créer un tutoriel abordable et facilement compréhensible pour tous et non de faire un cours d'algèbre linéaire.<br><br>
- **Pour plus d'informations :**
    - [Page Wikipedia Régression linéaire](https://fr.wikipedia.org/wiki/R%C3%A9gression_lin%C3%A9aire)
    - [Page Wikipedia Régression (statistiques)](https://fr.wikipedia.org/wiki/R%C3%A9gression_(statistiques))

### Hypothèses et propriétés de bases :

- Soit *Xi* les variables explicatives et *Y* la variable expliquée :
    - La régression linéaire repose sur l'hypothèse qu'il existe une relation linéaire entre la variable *expliquée Y* et les variables *explicatives Xi*.
    - Dans sa forme la plus simple, cette relation est décrite par une droite : *Y = aXi + b + ϵ*,\
où *a* est la pente de la droite, *b* l'ordonnée à l'origine, et *ϵ* un terme d'erreur représentant les écarts aléatoires par rapport à la droite.<br><br>
- La régression linéaire cherche à trouver les valeurs des paramètres *a* et *b* qui minimisent l'écart entre les prédictions du modèle et les valeurs observées de *Y*.
Cela se fait généralement en minimisant la somme des carrés des erreurs.\
Exemple de méthode des moindres carrés avec OLS (Ordinary Least Squares regression) :
$$
\text{SSE} = \sum_{i=1}^{n} (Y_{\text{observé}, i} - Y_{\text{prédit}, i})^2
$$
    - où :
        - *Yobservé* : est la valeur observée réelle de la variable expliquée pour le *i*-ème point de données,
        - *Yprédit* : est la valeur prédite de la variable expliquée pour le *i*-ème point de données,
        - *n* : est le nombre total d'observations.<br><br>
- Hypothèses à vérifier pour que le modèle soit significatif et pertinent:
    - Non colinéarité des variables explicatives
    - Indépendance des erreurs
    - Exogénéité
    - Homoscédasticité
    - Normalité des termes d'erreur<br><br>

## Réalisation d'une régression linéaire avec les deux bibliothèques :

- On va choisir trois variables quantitatives pertinentes et réaliser une régression linéaire avec les deux bibliothèques, entre les variables deux par deux (la première vs la deuxième, la deuxième vs la troisième, la troisième vs la première) et représenter les résultats graphiquement.<br><br>
- On choisit les variables suivantes pour des pertinences biologiques et statistiques :
    - Culmen Length (mm) soit la longueur du culmen (bec)
    - Flipper Length (mm) soit la longueur des nageoires
    - Body Mass (g) soit la masse corporelle.<br><br>
- En effet, ces trois variables englobent des indicateurs clés sur l'adaptation et le comportement des pingouins et semblent être corrélées. Par exemple, les pingouins plus grands, donc qui ont une longueur de bec et de nageoire plus grande, ont tendance à avoir une masse corporelle plus élevée. Ainsi, ces variables présentent une forte variance au sein des données et sont donc pertinentes pour une analyse statistique. Par ailleurs, dans le contexte actuel du changement climatique et des catastrophes naturelles, il est intéressant de regarder les relations entre les caractères morphologiques des pingouins, notamment pour mieux comprendre comment ceux-ci s'adaptent à leur environnement dans un contexte écologique et de conservation des espèces.<br><br>
- Aucune variable commune entre les deux exemples ne sera utilisée pour bien expliciter la manière d'utiliser ces deux bibliothèques de manière claire et limpide (*i.e.* les variables utilisées seront redéfinies à chaque fois).

### Avec scikit-learn, bibliothèque plutôt utilisée pour le machine learning :

In [ ]:
# Sélection des variables quantitatives pour les modèles de régression.
# y1 représente la longueur du bec et x1 la longueur de la nageoire.
y1 = df['Culmen Length (mm)']
x1 = df['Flipper Length (mm)']

# y2 représente la longueur de la nageoire et x2 la masse corporelle.
y2 = df['Flipper Length (mm)']
x2 = df['Body Mass (g)']

# y3 représente la masse corporelle et x3 la longueur du bec.
y3 = df['Body Mass (g)']
x3 = df['Culmen Length (mm)']


# Création d'une fonction pour tracer les données et la ligne de régression.
def plot_regression(x, y, model, x_label, y_label):
    """
    Trace les données observées et la ligne de régression.

    Parameters
    ----------
    x : pd.Series
        La variable explicative.
    y : pd.Series
        La variable expliquée.
    model : sk.linear_model.LinearRegression
        Le modèle de régression ajusté.
    x_label : str
        L'étiquette pour l'axe des x.
    y_label : str
        L'étiquette pour l'axe des y.
    """
    # Création d'une figure pour le tracé.
    plt.figure(figsize=(8, 5))
    # Tracer un nuage de points pour les données observées en bleu.
    plt.scatter(x, y, color='pink', label='Données observées')
    # Tracer la ligne de régression en rouge.
    # Utilise .reshape(-1, 1) car le modèle attend un tableau 2D pour x.
    # .predict() prédit les valeurs de y en fonction de x.
    plt.plot(
        x,
        model.predict(x.values.reshape(-1, 1)),
        color='blue',
        label='Ligne de régression'
    )
    # Ajouter des étiquettes pour les axes x et y.
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    # Ajouter une légende et une grille au graphique.
    plt.legend()
    plt.grid()
    # Ajouter un titre au graphique.
    plt.title(f"Ligne de régression : {y_label} vs {x_label}")
    # Afficher le graphique.
    plt.show()


# Création et ajustement du modèle 1 pour les variables x1 et y1.
model1 = sk.linear_model.LinearRegression()
x1_reshaped = x1.values.reshape(-1, 1)  # Mise en forme en 2D pour le modèle.
model1.fit(x1_reshaped, y1)  # Ajuste le modèle aux données.

# Création et ajustement du modèle 2 pour les variables x2 et y2.
model2 = sk.linear_model.LinearRegression()
x2_reshaped = x2.values.reshape(-1, 1)  # Mise en forme en 2D pour le modèle.
model2.fit(x2_reshaped, y2)  # Ajuste le modèle aux données.

# Création et ajustement du modèle 3 pour les variables x3 et y3.
model3 = sk.linear_model.LinearRegression()
x3_reshaped = x3.values.reshape(-1, 1)  # Mise en forme en 2D pour le modèle.
model3.fit(x3_reshaped, y3)  # Ajuste le modèle aux données.

# Affichage des résultats pour chaque modèle.
# (coefficient, intercept, MSE et R²).
for i, (model, x, y) in enumerate(
    [(model1, x1, y1),
     (model2, x2, y2),
     (model3, x3, y3)], start=1):
    # Calcul de l'erreur quadratique moyenne (MSE)
    # et du R² pour chaque modèle.
    mse = sk.metrics.mean_squared_error(
        y, model.predict(x.values.reshape(-1, 1)))
    r_squared = sk.metrics.r2_score(
        y, model.predict(x.values.reshape(-1, 1)))
    # Affiche les résultats du modèle : coefficient, intercept, MSE et R².
    print(f"\nModèle {i} :")
    print(f"Coefficient de régression : {model.coef_[0]}")
    print(f"Ordonnée à l'origine : {model.intercept_}")
    print(f"MSE : {mse}\nR²  : {r_squared}")

# Tracer des résultats pour chaque modèle avec des étiquettes d'axes.
plot_regression(
    x=x1, y=y1, model=model1,
    x_label='Longueur de la nageoire (mm)',
    y_label='Longueur du bec (mm)')

plot_regression(
    x=x2, y=y2, model=model2,
    x_label='Masse corporelle (g)',
    y_label='Longueur de la nageoire (mm)')

plot_regression(
    x=x3, y=y3, model=model3,
    x_label='Longueur du bec (mm)',
    y_label='Masse corporelle (g)')

### Avec statmodels, bibliothèque plutôt utilisée pour les études statistiques classiques :

In [ ]:
# Création d'une fonction pour effectuer une régression linéaire.
def perform_regression(x, y):
    """
    Effectue une régression linéaire entre deux variables.

    Parameters
    ----------
    x : pd.Series
        La variable explicative.
    y : pd.Series
        La variable expliquée.

    Returns
    -------
    statsmodels.regression.linear_model.RegressionResultsWrapper
        Résultats du modèle de régression linéaire ajusté.
    """
    # Ajouter une constante pour inclure
    # une ordonnée à l'origine (interception) dans le modèle.
    X = sm.add_constant(x)
    # Ajuster un modèle de régression linéaire en utilisant
    # la méthode des moindres carrés (Ordinary Least Squares - OLS)
    # .fit() ajuste le modèle pour y et X afin de minimiser
    # les erreurs.
    model = sm.OLS(y, X).fit()
    # Retourne l'objet contenant les résultats du modèle ajusté.
    return model


# Création d'une fonction pour visualiser la régression avec un graphique.
def plot_regression(x, y, model, x_label, y_label):
    """
    Trace les données observées et la ligne de régression.

    Parameters
    ----------
    x : pd.Series
        La variable explicative.
    y : pd.Series
        La variable expliquée.
    model : statsmodels.regression.linear_model.RegressionResultsWrapper
        Le modèle de régression ajusté.
    x_label : str
        L'étiquette pour l'axe des x.
    y_label : str
        L'étiquette pour l'axe des y.
    """
    # Créer une figure pour le graphique de régression
    # avec une taille spécifiée.
    plt.figure(figsize=(8, 5))
    # Tracer les données en tant que nuage de points en bleu.
    plt.scatter(x, y, color='pink', label='Données observées')
    # Tracer la ligne de régression en rouge,
    # avec les prédictions du modèle ajusté.
    # .predict() prédit les valeurs de y en fonction de x.
    plt.plot(
        x,
        model.predict(sm.add_constant(x)),
        color='blue',
        label='Ligne de régression'
    )
    # Ajouter des étiquettes aux axes x et y.
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    # Titre du graphique qui précise les variables étudiées.
    plt.title(f'Ligne de régression : {y_label} vs {x_label}')
    # Ajouter une légende pour expliquer les éléments du graphique.
    plt.legend()
    # Ajouter une grille au graphique.
    plt.grid()
    # Afficher le graphique.
    plt.show()

# Sélectionner des variables quantitatives.
# Variables pour la première régression.


y1 = df['Culmen Length (mm)']  # Longueur du bec en mm.
x1 = df['Flipper Length (mm)']  # Longueur de la nageoire en mm.

# Variables pour la deuxième régression.
y2 = df['Flipper Length (mm)']  # Longueur de la nageoire en mm.
x2 = df['Body Mass (g)']        # Masse corporelle en grammes.

# Variables pour la troisième régression.
y3 = df['Body Mass (g)']         # Masse corporelle en grammes.
x3 = df['Culmen Length (mm)']    # Longueur du bec en mm.

# Effectuer les régressions linéaires
# pour chaque paire de variables.
model1 = perform_regression(x1, y1)  # Première régression.
model2 = perform_regression(x2, y2)  # Deuxième régression.
model3 = perform_regression(x3, y3)  # Troisième régression.

# Afficher les résumés des modèles et l'erreur quadratique moyenne (MSE)
# pour chaque régression.
print("\nModèle 1 Résumé :")
# Résumé des statistiques du modèle de la première régression.
print(model1.summary())
# Erreur quadratique moyenne (MSE) pour la première régression.
print("MSE :", model1.mse_total)

print("\nModèle 2 Résumé :")
# Résumé des statistiques du modèle de la deuxième régression.
print(model2.summary())
# Erreur quadratique moyenne (MSE) pour la deuxième régression.
print("MSE :", model2.mse_total)

print("\nModèle 3 Résumé :")
# Résumé des statistiques du modèle de la troisième régression.
print(model3.summary())
# Erreur quadratique moyenne (MSE) pour la troisième régression.
print("MSE :", model3.mse_total)

# Tracer les résultats de chaque régression.
plot_regression(
    x1, y1, model1,
    'Longueur de la nageoire (mm)',  # Étiquette pour l'axe x
    'Longueur du bec (mm)'           # Étiquette pour l'axe y
)
plot_regression(
    x2, y2, model2,
    'Masse corporelle (g)',          # Étiquette pour l'axe x
    'Longueur de la nageoire (mm)'   # Étiquette pour l'axe y
)
plot_regression(
    x3, y3, model3,
    'Longueur du bec (mm)',          # Étiquette pour l'axe x
    'Masse corporelle (g)'           # Étiquette pour l'axe y
)

### Comparaison entre les deux :

- On peut d'ores et déjà remarquer que statmodels est plus flexible que scikit-learn. En effet, scikit-learn nécessite des tableaux 2D (matrice ou array) tandis que statmodels ne requiert aucun formatage particulier de la sorte. Cela est dû au fait que pour ses capacités de machine learning, scikit-learn a besoin d'être efficace et de pouvoir gérer de grands ensembles de données efficacement et automatiquement ainsi que de gérer de nombreuses opérations de scaling. Ce formatage permet également un prétraitement uniforme des données facilité (normalisation, standardisation, ...).<br><br>
- statmodels semble plus clair au niveau interprétabilité, à directement et facilement afficher le résumé de l'analyse ainsi que ses descripteurs usuels pour l'évaluation de celle-ci (critères d'évaluation). scikit-learn ne le fait pas aussi facilement, car dans des contextes de machine learning l'utilisateur doit pouvoir évaluer son modèle selon des besoins bien spécifiques.<br><br>
- On remarque que statmodels fournit une analyse plus approfondie des résultats tels que des intervalles de confiance (ressemble à R dans le formatage), même si cela est aussi possible avec scikit-learn quoique plus sommaire et non directement disponible. Par contre, on remarque que statmodels indique si le modèle obtenu présente des erreurs potentielles, comme vu dans les messages affichés par le résumé, afin d'affiner les modèles obtenus et de permettre une analyse statistique des données complète.<br><br>
- On observe une adéquation parfaite à l'œil  nu entre les représentations graphiques des deux modèles. Regardons maintenant les descripteurs pour l'évaluation de la qualité du modèle afin d'affiner notre comparaison.

## Critères d'évaluations basiques importants et détails

- **R² (coefficient de détermination) :**
    - Il mesure la proportion de la variance de la variable dépendante expliquée par le modèle. Son score varie entre 0 et 1, où une valeur proche de 1 indique que le modèle explique bien les variations des données, tandis qu’une valeur proche de 0 suggère que le modèle explique peu de variabilité. "R-squared" sous statmodels dans le summary et ".r2_score()" (dans le code, sinon affiché comme "R²" avec les print) sous scikit.
    $$
R^2 = 1 - \frac{\text{SSE}}{\text{SST}}
$$

où :
  - **SSE** est la somme des carrés des erreurs, donnée par :
  
    $$
    \text{SSE} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
    $$
  
  - **SST** est la somme des carrés totaux par rapport à la moyenne des valeurs observées :
  
    $$
    \text{SST} = \sum_{i=1}^{n} (y_i - \bar{y})^2
    $$

- **MSE** (erreur quadratique moyenne) :
    - Elle évalue l’erreur moyenne de prédiction du modèle, en prenant la moyenne des carrés des erreurs entre les valeurs observées et prédites. Plus le MSE est faible (proche de 0), plus le modèle est précis. ".mse_total" sous statmodels (dans le code,  sinon affiché comme "MSE" avec les print)  et "mean_squared_error()" (dans le code, sinon affiché comme "MSE" avec les print) sous scikit.
    $$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

où :
  - *yi* : est la valeur observée,
  - *ŷi* : est la valeur prédite par le modèle,
  - *n* : est le nombre de données.<br><br>

- **Coefficient de régression (pente de la courbe) :**
  - Le coefficient (ou pente) indique l'effet du changement de la variable explicative *Xi* sur la variable expliquée *Y*. Un coefficient positif signifie que lorsque *Xi* augmente, *Y* tend également à augmenter, tandis qu'un coefficient négatif signifie que *Y* diminue lorsque *Xi* augmente. "coef" sous statmodels dans le summary et "model.coef_" (dans le code sinon affiché comme "Coefficient de régression" avec les print) sous scikit.<br><br>

- **Ordonnée à l'origine :**
  - L'ordonnée à l’origine représente la valeur de *Y* lorsque *Xi* = 0. Cette valeur peut être interprétée comme la base à partir de laquelle *Y* évolue en fonction de *Xi*. Son interprétation est dépendante du contexte, elle peut ne pas être très pertinente en soi. Premier paramètre de ".params" dans statmodels (n'a pas été affiché par souci de lisibilité) et "model.intercept" (dans le code, sinon affiché comme "Ordonnée à l'origine" avec les print) sous scikit.<br><br>
 
- **Brève conclusion vis-à-vis de la régression linéaire réalisée :**
    - **Tous les R² sont >0, indiquant donc une corrélation entre chaque paire de variables. Celle-ci est plus prononcée (car plus proche de 1, 0.7) pour la seconde paire de variables "Flipper Length (mm) vs Body Mass (g)".**<br><br>
    - **Néanmoins, on observe que les MSE sont relativement voire très élevées** (654 930 pour la troisième paire de variables), cela pourrait être acceptable si les valeurs étaient très élevées elles aussi mais ce n'est pas le cas. La plus petite MSE est de 17 pour la première paire de variables ce qui pourrait être acceptable si on considère que c'est proche de 0...<br><br>
    - **En somme, la régression linéaire montre des corrélations prononcées entre les variables, notamment "Flipper Length (mm) vs Body Mass (g)", mais la proximité des prédictions avec la réalité semble fortement compromise par les valeurs élevées des MSE.<br><br>**
    - **Lien avec les signalements de statmodels, multicolinéarité et le fait que les valeurs ne sont ni centrées ni réduites (non normalisées) : en somme problème d'échelles --> voir la suite.<br><br>**
- **Pour plus d'informations sur les autres descripteurs affichés :**
    - [Doc. statmodels](https://www.statsmodels.org/devel/index.html)
    - [Le Summary de statmodels et autres](https://blog.dailydoseofds.com/p/statsmodel-regression-summary-will)
    - [Doc. scikit](https://scikit-learn.org/stable/)<br><br>

- **Détails :**
    - Les R² obtenus sont les mêmes, ce qui montre bien que les deux bibliothèques sont aussi efficaces. Néanmoins, ce n'est pas le cas pour les MSE mais cela est sûrement dû à une différence dans leur définition et/ou affichage entre les deux bibliothèques. Elles pourraient ne pas être comparables sur ce point là. Mais comme les MSE calculées par scikit sont plus faibles à chaque fois que celles calculées par statmodels (même si du même ordre de grandeur), on peut aussi dire que scikit affine davantage la prédiction des résultats à la réalité que statmodels (différence entre machine learning et statistiques classiques). On remarque que le modèle 2 explique le plus de variance, même si comme le dit la seconde note du résumé (comme pour le modèle 1) de statmodels, il semble y avoir un problème de multicolinéarité des variables.<br><br>
    - La multicolinéarité se produit lorsque deux ou plusieurs variables explicatives dans un modèle de régression sont fortement corrélées entre elles. Cela est problématique, notamment parce qu’elle rend l’estimation des coefficients de régression instable et réduit donc la fiabilité des résultats. En effet, car cela empêche le modèle de déterminer l'influence indépendante de chaque variable explicative.<br><br>
    - La multicolinéarité complique par conséquent l’interprétation des coefficients. Normalement, un coefficient dans une régression linéaire représente l’effet marginal de la variable explicative sur la variable dépendante, toutes les autres variables étant maintenues constantes. Mais lorsque deux variables sont très corrélées, cet effet marginal perd de son sens car il est difficile de faire varier une variable sans affecter l’autre.<br><br>
    - La première note du résumé de statmodels indique que les erreurs-types calculées reposent sur l’hypothèse que la matrice de covariance des erreurs est correctement spécifiée. Si le modèle est un premier test ce n'est pas un problème majeur, mais il est effectivement  important d’en tenir compte car une matrice mal spécifiée peut entraîner des erreurs d'interprétation des coefficients.<br><br>
    - Ce faisant, on rejoint donc la différence entre statmodels et scikit-learn au niveau de la flexibilité : scikit-learn est moins flexible et impose un tableau 2D mais cela est fait pour optimiser ses performances, là où statmodels attend que l'utilisateur sache ce qu'il fait mais affiche en plus des conseils et signalements si certaines conditions ne sont pas respectées (*i.e*  hypothèses à vérifier pour que le modèle soit significatif et pertinent).<br><br>
    - **Dans notre cas, le signalement automatique 1 de statmodels est pertinent car nous n'avons pas vérifié les conditions de validité comme le but du projet est de présenter les deux librairies et non de faire une étude statistique poussée et exhaustive avec hypothèses à l'appui. Le signalement 2 montre qu'il y'a un problème au niveau de l'échelle des données, les données ont donc peut-être une trop grande variabilité de base. On remarque que nous n'avons à aucun moment centré et/ou réduit les données (*i.e.* normalisé) ou utilisé un z-score ce qui pose nécessairement problème comme on compare des valeurs en "mm" à des valeurs en "g", causant un problème d'échelle et donc sûrement le signalement 2.<br><br>**
    - **Formule du z-score (normalisation standard):**

$$
z = \frac{x - \mu}{\sigma}
$$
où :
- z : Le z-score (valeur normalisée)
- x : La valeur initiale
- $\mu$ : La moyenne de l'ensemble des données
- $\sigma$ : L'écart-type de l'ensemble des données

#### Modèle 2 corrigé, soit *Flipper Length (mm)* vs *Body Mass (g)* :

In [ ]:
y = df['Flipper Length (mm)']
x = df['Body Mass (g)']

# Standardisation des données.
x_scaled = (x - x.mean()) / x.std()
y_scaled = (y - y.mean()) / y.std()

# Ajout de la constante.
X_scaled = sm.add_constant(x_scaled)

# Ajustement du modèle.
model_scaled = sm.OLS(y_scaled, X_scaled).fit()

# Prédictions.
y_pred = model_scaled.predict(X_scaled)

# Afficher le résumé.
print(model_scaled.summary())
# Erreur quadratique moyenne (MSE).
print("MSE :", model_scaled.mse_total)

# Tracer les résultats.
plt.figure(figsize=(8, 5))
plt.scatter(x_scaled, y_scaled, color='pink', label='Données observées')
plt.plot(x_scaled, y_pred, color='blue', label='Ligne de régression')
plt.xlabel('Longueur de la nageoire (mm) standardisée')
plt.ylabel('Longueur du bec (mm) standardisée')
plt.title('Ligne de régression : Longueur du bec vs Longueur de la nageoire')
plt.legend()
plt.grid()
plt.show()

- **Le signalement 2 du résumé de statmodels n'apparaît plus, néanmoins le modèle 1 comparait la même unité et affichait tout de même le signalement là où celui-ci n'apparaissait pas dans le modèle 3 malgré deux unités différentes. Ce faisant, les données doivent aussi présenter une très grande variabilité de base. Cela pourrait aussi indiquer que les variables dans le modèle 1 sont plus fortement corrélées que celles du modèle 3.<br><br>**
- **Par contre, on remarque que le MSE est passé de 194 à 1 entre le modèle fait précédemment et celui-ci qui est corrigé (mis à l'échelle). Ce faisant, on peut conclure que le problème vu précédemment vis-à-vis de la précision des prédictions établies était dû à l'absence de mise à l'échelle !<br><br>**
- **On pourrait (et devrait dans une analyse) vérifier que c'est bien le cas pour la troisième paire de variables mais l'absence de mise à l'échelle était volontaire et à but pédagogique pour expliciter davantage l'utilisation des bibliothèques et les erreurs possibles. Ce faisant, on considérera, la partie régression linéaire comme étant finie. À vous de jouer !<br><br>**

# Principe de l'ACP (Analyse par Composantes Principales) ou PCA en anglais :

### Définition :

L'ACP est une technique en statistique qui permet de réduire les dimensionnalités d'un ensemble de données d'une base à une autre plus petite tout en conservant le maximum d'informations. Elle permet de transformer des données corrélées en un ensemble de variables indépendantes, appelées composantes principales, qui sont ordonnées de façon à expliquer au mieux la
variance des données d'origine : la première composante principale capte la plus grande part de variation, la seconde représente la plus grande part restante, et ainsi de suite. Pour choisir le bon nombre de composantes pour l'ACP, il faut prendre une valeur de composantes qui expliquent un pourcentage satisfaisant de la variance totale (80 % par exemple). Pour cela, on peut utiliser la "méthode du coude" qui permet d'identifier graphiquement le point à partir duquel le gain de variance n'est pas très important. <br>
L'ACP permet ainsi d'analyser plusieurs variables simultanément, et de voir d'éventuelles relations "cachées" dans les données, autrement dit, de trouver des regroupements qu'on n'aurait pas forcément vus au premier abord.\
L'ACP est une technique venant de la physique, un exemple simple pour mieux la comprendre est la photographie : une photo transforme une vision en 3D en une photo sur un plan 2D (les photos montrent bien les distances mais mal le relief en général, car elles ont perdu la profondeur comme dimension).

## Les différentes étapes pour réaliser une ACP :


- **Étape 1 - Normalisation des variables :** On place les données sur des échelles comparables pour éviter que des variables à grande variance dominent l’analyse.

- **Étape 2 - Création d'une matrice de covariance :** Permet d’identifier les corrélations entre les variables, en calculant une matrice qui montre comment chaque paire de variables est liée.

- **Étape 3 - Identification des composantes principales :** Décomposer la matrice  pour trouver les valeurs propres  (quantité de variance expliquée par chaque composante) et vecteurs propres (représentent la direction des composantes principales, qui sont orthogonaux).

- **Étape 4 - Sélection des caractéristiques :** En fonction de l’importance des composantes, on choisit celles à conserver pour constituer un vecteur des caractéristiques, favorisant la réduction de dimensions.

- **Étape 5 - Projection des données :** Enfin, les données sont projetées sur les axes des composantes principales pour créer un espace réduit qui conserve l’essentiel de l’information.<br><br>

Importation des sous bibilothèques pour scikit-learn et statsmodels :

In [ ]:
from sklearn.decomposition import PCA
# pour normaliser le jeu de données.
from sklearn.preprocessing import StandardScaler
from statsmodels.multivariate.pca import PCA as StatsmodelsPCA
from sklearn.impute import SimpleImputer

Ensemble des caractéristiques des différentes variables :


In [ ]:
# Analyse descriptive du jeu de donées.
df.describe()

# ACP - code commun aux deux bibliothèques :

## Visualisation de la heatmap de la matrice de corrélation :

In [ ]:
# Chargement des données.
data = pd.read_csv("penguins.csv")

# Sélection des variables quantitatives.
quantitative_data = data[['bill_length_mm', 'bill_depth_mm', 
                           'flipper_length_mm', 'body_mass_g']]

# Imputation des valeurs manquantes avec la moyenne des colonnes.
imputer = SimpleImputer(strategy="mean")
quantitative_data_imputed = imputer.fit_transform(quantitative_data)

# Normalisation des données.
scaler = StandardScaler()
quantitative_data_normalized = scaler.fit_transform(quantitative_data_imputed)

# Visualisation de la heatmap de corrélation.
correlation_matrix = pd.DataFrame(quantitative_data_normalized,
                                  columns=quantitative_data.columns).corr()
plt.figure(figsize=(8, 6))
plt.title("Heatmap de la matrice de corrélation")
plt.imshow(correlation_matrix, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar()

# Ajout des valeurs numériques dans chaque case de la heatmap.
for i in range(len(correlation_matrix)):
    for j in range(len(correlation_matrix.columns)):
        plt.text(j, i, f"{correlation_matrix.iloc[i, j]:.2f}",
                 ha="center", va="center", color="black")

plt.xticks(range(len(correlation_matrix.columns)),
           correlation_matrix.columns, rotation=45)
plt.yticks(range(len(correlation_matrix.columns)),
           correlation_matrix.columns)
plt.show()

### Corrélations entre les variables :

La **heatmap de la matrice de corrélation** est une représentation graphique qui illustre les relations entre les différentes variables d’un jeu de données. Chaque cellule de cette matrice est colorée en fonction de la force et du sens de la corrélation entre deux variables, selon une échelle de couleur (ici, rouge pour des corrélations positives et bleu pour des corrélations négatives).

Dans ce cas, la **heatmap** met en évidence des relations significatives entre les variables. Par exemple, on observe que les variables *body_mass_g* (masse corporelle) et *flipper_length_mm* (longueur des nageoires) sont fortement et positivement corrélées, indiquant qu'une augmentation de l'une est souvent associée à une augmentation de l'autre. En revanche, d'autres combinaisons de variables montrent des corrélations plus faibles, suggérant une indépendance relative entre elles.

Le but de la **heatmap** est de visualiser rapidement ces relations et de détecter des redondances entre les variables. Ces redondances signifient que certaines dimensions des données contiennent des informations similaires, ce qui justifie l’utilisation d’une méthode de réduction de dimensionnalité, comme l’Analyse en Composantes Principales (ACP), pour synthétiser ces relations et simplifier l’analyse tout en conservant l'essentiel des informations.


## Tableau de corrélation :

In [ ]:
# Calcul du tableau de corrélation brut (avant normalisation).
correlation_matrix = quantitative_data.corr()

# Affichage du tableau de corrélation.
print("Tableau de corrélation :")
print(correlation_matrix)

La fonction *quantitative_data.corr()* génère une matrice de corrélation qui quantifie la relation linéaire entre chaque paire de variables. Les valeurs obtenues vont de -1 (corrélation négative parfaite) à +1 (corrélation positive parfaite), avec 0 indiquant une absence de  corrélation. 

# ACP avec scikit-learn :

## Variances expliquées et projection de l'ACP :

In [ ]:
# Ajout des étiquettes.
labels = data['species']

# Application de l'ACP avec Scikit-learn.
pca = PCA(n_components=2)
components = pca.fit_transform(quantitative_data_normalized)
explained_variance = pca.explained_variance_ratio_

# Affichage de la variance expliquée.
print("Variance expliquée par les composantes principales :")
for i, var in enumerate(explained_variance[:2], 1):
    print(f"Composante {i} : {var:.2%}")

# Affichage des résultats de l'ACP sous forme de graphique simple.
plt.figure(figsize=(8, 6))
species_colors = labels.map({'Adelie': 'blue', 'Gentoo': 'red', 'Chinstrap': 'green'})
scatter = plt.scatter(components[:, 0], components[:, 1], 
                      c=species_colors, edgecolor="k", s=50)
plt.title("Projection ACP avec scikit-learn (2 premières composantes)")
plt.xlabel(f"Composante 1 ({explained_variance[0]:.2%} variance expliquée)")
plt.ylabel(f"Composante 2 ({explained_variance[1]:.2%} variance expliquée)")

# Ajout de la légende.
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=color, markersize=10, 
                             label=species)
                  for species, color in zip(labels.unique(), ['blue', 'red', 'green'])]
plt.legend(handles=legend_handles)
plt.grid()
plt.show()

#### Variance expliquée :

- La première composante principale explique environ **68,84 %** de la variance totale des données, ce qui indique qu'elle capte la majorité de l'information. Cela suggère qu'il existe une direction principale dans l'espace des variables où les observations sont fortement dispersées.
- La deuxième composante principale explique environ **19,31 %** de la variance totale. Bien qu'elle soit moins informative que la première, elle permet de capturer des variations complémentaires. <br><br>
Ensemble, ces deux composantes expliquent **88,15 %** de la variance totale, ce qui signifie qu'une représentation bidimensionnelle est suffisante pour décrire la structure globale des données avec une bonne précision.

#### Projection  de l'ACP :

Le graphique de projection montre les trois espèces de manchots : Adélie (bleu), Gentoo (rouge), et Chinstrap (vert), projetées sur les deux premières composantes principales. Voici l'interprétation :

- Adélie (bleu) : Les points bleus sont regroupés principalement dans la partie inférieure gauche du graphique, indiquant une certaine homogénéité au sein de cette espèce sur les deux premières composantes principales.

- Chinstrap (vert) : Les points verts se situent légèrement au-dessus des points bleus en les chevauchant partiellement, mais ils sont bien distingués des points rouges (Gentoo). 

- Gentoo (rouge) : Les points rouges sont concentrés dans la partie droite du graphique, avec une plus grande dispersion sur la composante principale 1. Cela suggère que cette espèce est plus variable que les autres en termes de morphologie, mais reste clairement séparée des autres espèces sur la projection.


En résumé, le graphique montre une séparation claire entre les Gentoo et les deux autres espèces (Adélie et Chinstrap),qui présentent un chevauchement entre elles. Cela suggère que, bien que certaines caractéristiques morphologiques puissent différencier les espèces,  Adélie et Chinstrap partagent des caractéristiques similaires sur les deux premières composantes principales, tandis que Gentoo forme un groupe distinct.

#### Interprétation des résultats :

La première composante semble être associée à une variation générale de taille corporelle (variables comme *"body_mass_g"* et *"flipper_length_mm"*), capturant des différences globales entre les espèces.
La deuxième composante pourrait refléter des variations plus subtiles liées à des proportions ou des caractéristiques spécifiques comme la relation entre la longueur et la profondeur du bec.

## ACP avec statsmodels :

In [ ]:
# Chargement des données.
data = pd.read_csv("penguins.csv")

# Sélection des variables quantitatives et des étiquettes.
quantitative_data = data[['bill_length_mm', 'bill_depth_mm', 
                           'flipper_length_mm', 'body_mass_g']]
labels = data['species']

# Imputation des valeurs manquantes avec la moyenne des colonnes.
imputer = SimpleImputer(strategy="mean")
quantitative_data_imputed = imputer.fit_transform(quantitative_data)

# Normalisation des données.
scaler = StandardScaler()
quantitative_data_normalized = scaler.fit_transform(quantitative_data_imputed)

# Application de l'ACP avec Statsmodels.
# Statsmodels PCA nécessite des données normalisées.
pca = sm.multivariate.PCA(quantitative_data_normalized, ncomp=2)

# Extraction des résultats de l'ACP.
# Composantes principales.
pca_components = pca.factors
# Variance expliquée par chaque composante.
explained_variance = pca.eigenvals / sum(pca.eigenvals)

# Affichage de la variance expliquée.
print("Variance expliquée par les composantes principales :")
for i, var in enumerate(explained_variance, 1):
    print(f"Composante {i} : {var:.2%}")

# Visualisation de la projection des données sur les 2 premières composantes.
plt.figure(figsize=(8, 6))
species_colors = labels.map({'Adelie': 'blue', 'Gentoo': 'red', 'Chinstrap': 'green'})
plt.scatter(pca_components[:, 0], pca_components[:, 1], 
            c=species_colors, edgecolor="k", s=50)

# Titres et étiquettes.
plt.title("Projection ACP avec statsmodels (2 premières composantes)")
plt.xlabel(f"Composante 1 ({explained_variance[0]:.2%} variance expliquée)")
plt.ylabel(f"Composante 2 ({explained_variance[1]:.2%} variance expliquée)")

# Ajout de la légende.
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=color, markersize=10, 
                             label=species)
                  for species, color in zip(labels.unique(), ['blue', 'red', 'green'])]
plt.legend(handles=legend_handles)
plt.grid()
plt.show()

#### Projection  de l'ACP :

Sur le graphique ci-dessus, on peut voir :
- Les points rouges (Gentoo) qui sont séparés des deux autres groupes et sont bien distants sur l'axe de la composante 1. Cette séparation nette des Gentoo suggère que cette espèce a des caractéristiques morphologiques distinctes qui permettent une identification facile par l'ACP.

- Les points bleus (Adelie) et verts (Chinstrap) montrent un chevauchement important, particulièrement sur la composante 1 (axe horizontal). Cela indique que ces deux espèces partagent des caractéristiques similaires et que l'ACP ne les sépare pas aussi efficacement que les Gentoo.

Le chevauchement des groupes bleus et verts indique que la différence entre ces deux espèces est plus subtile et qu'il pourrait y avoir un chevauchement plus marqué dans les traits morphologiques. Cela pourrait être dû à une variabilité plus faible au sein de ces espèces, ou à des traits morphologiques qui se ressemblent davantage entre elles comparées aux Gentoo.

## Comparaison entre les deux bibliothèques :

- Avec **statsmodels**, nous avons obtenu une projection des données sur les deux premières composantes principales, qui expliquent respectivement **78,09 %** et **21,91 %** de la variance. Le graphique issu de cette analyse montre une séparation nette des groupes, en particulier pour les Gentoo, qui sont distinctement séparés des autres groupes. Cependant, Adelie (en bleu) et Chinstrap (en vert) présentent un chevauchement important, ce qui indique que ces deux espèces partagent des caractéristiques morphologiques similaires, rendant leur séparation plus difficile.<br>
Ces résultats suggèrent que la méthode peut être efficace pour réduire la dimensionnalité et repérer des distinctions marquées (comme pour les Gentoo), mais elle peut aussi avoir des limitations lorsqu'il s'agit de différencier des groupes qui se chevauchent ou sont plus proches dans l’espace des caractéristiques (Adelie et Chinstrap).

- Avec **scikit-learn**, l'ACP nous a permis de visualiser la répartition des espèces sur les deux premières composantes principales, où la composante 1 expliquait **68,84 %** de la variance et la composante 2 **19,31 %**. Le graphique résultant est assez similaire à celui de **statsmodels**, mais avec une légère différence d’orientation des groupes. Les Gentoo (rouges) sont de nouveau bien séparés, tandis que les Adélie et Chinstrap se chevauchent, tout comme dans l'analyse avec **statsmodels**. La différence principale réside dans l’orientation du graphique, mais cette variation ne change pas fondamentalement la conclusion sur les relations entre les espèces.

    - **Séparation des groupes** : Les deux bibliothèques montrent une bonne séparation des Gentoo (en rouge) des autres espèces, ce qui suggère que ces manchots possèdent des caractéristiques morphologiques suffisamment distinctes pour être identifiés facilement dans le cadre de cette analyse. Cependant, Adelie et Chinstrap se chevauchent dans les deux analyses, ce qui peut indiquer que ces deux espèces partagent des traits physiques similaires qui les rendent difficiles à distinguer sur les premières composantes principales.

    - **Différences dans l’orientation** : La principale différence entre les résultats des deux bibliothèques réside dans l'orientation des groupes. **Statsmodels** semble offrir une projection où les groupes sont légèrement inclinés différemment par rapport à **scikit-learn**, ce qui pourrait être dû à la manière dont les composantes principales sont extraites ou à des différences dans les paramètres par défaut des deux méthodes. En effet, normalement peu importe la méthode de calcul utilisée pour les ACP, les valeurs propres seront toujours les mêmes mais les vecteurs propres, eux, peuvent être différents. Comme ceux-ci déterminent l'orientation selon les axes des graphiques, des différences dans la base à deux dimensions peuvent apparaître (différences d'orientation). Mais peu importe l'orientation du graphique, les distances entre les clusters (groupes dans les données) seront les mêmes (penser à la valeur absolue en mathématiques). Ainsi, deux ACP peuvent donner deux graphiques différents qui disent exactement la même chose, il faut donc être vigilant !

    - **Précision des résultats** : Les deux méthodes fournissent des résultats similaires en termes de variance expliquée et de répartition des groupes, bien que **scikit-learn** soit souvent préféré dans le cadre d’analyses d’ACP en raison de sa flexibilité, de sa vitesse et de ses outils de visualisation plus puissants.

## Conclusion générale sur l'ACP :

Qu’elle soit réalisée avec **statsmodels** ou **scikit-learn**, l'ACP nous offre une vision claire de la structure des données et nous aide à comprendre les relations entre les différentes espèces de manchots. L'ACP a réussi à réduire la complexité des données tout en préservant leur structure essentielle. La distinction entre les espèces sur la projection bidimensionnelle valide la pertinence des variables choisies pour cette analyse. Les deux bibliothèques montrent que les Gentoo peuvent être facilement distingués des autres espèces, mais que les Adélie et Chinstrap se chevauchent, suggérant qu'il peut être difficile de les différencier uniquement à partir des variables mesurées (longueur et profondeur du bec, longueur des nageoires, masse corporelle).

En résumé, **scikit-learn** et **statsmodels** sont deux bibliothèques efficaces pour effectuer une ACP, présentant des petites différences dans les résultats, notamment dans l'orientation des projections. **Scikit-learn** est davantage utilisé dans le milieu de l'analyse de données en raison de sa grande souplesse et de sa simplicité d'utilisation, tandis que **statsmodels** peut offrir des résultats similaires, mais avec une approche davantage axée sur la statistique.

## Autre possibilité de représentation graphique pour l'ACP (avec statsmodels) :

Il est également possible de visualiser une ACP sous forme de **cercle de corrélation** en utilisant une méthode appelée la décomposition en valeurs propres. Cette analyse est effectuée après avoir sélectionné uniquement les variables numériques (telles que la longueur et la profondeur du bec, la longueur des nageoires, et la masse corporelle des manchots). L'objectif principal est de comprendre les relations entre ces différentes variables à l’aide de la matrice de corrélation, qui quantifie les liens linéaires entre elles. En effectuant la décomposition en valeurs propres de cette matrice, nous obtenons des vecteurs propres (ou directions principales), qui sont ensuite visualisés à l’aide d’un cercle des corrélations, permettant d’illustrer graphiquement l’orientation et l’importance des relations entre les variables.

#### Cercle de corrélation juste avec statsmodels :

In [ ]:
# Sélectionner uniquement les colonnes numériques.
quantitative_data = data[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]

# Calculer la matrice de corrélation des variables.
corr_matrix = quantitative_data.corr()

# Effectuer la décomposition en valeurs propres.
eigvals, eigvecs = np.linalg.eig(corr_matrix)

# Trier les valeurs propres et les vecteurs propres par ordre décroissant.
sorted_indices = np.argsort(eigvals)[::-1]
eigvals = eigvals[sorted_indices]
eigvecs = eigvecs[:, sorted_indices]

# Créer un graphique de cercle des corrélations avec des couleurs et annotations.
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)

# Tracer le cercle.
circle = plt.Circle((0, 0),
                    1,
                    color='gray',
                    fill=False,
                    linestyle='--')
ax.add_artist(circle)

# Couleurs pour chaque vecteur propre (pour 4 variables).
colors = ['red', 'blue', 'green', 'purple']

# Tracer les vecteurs propres (les directions principales) avec couleurs et annotations.
quantitative_columns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
# Utiliser uniquement les colonnes quantitatives.
for i, var in enumerate(quantitative_columns):
    # Vérification de la dimension pour éviter l'IndexError.
    if i < eigvecs.shape[1]:
        ax.quiver(0, 0,
                  eigvecs[0, i],
                  eigvecs[1, i],
                  angles='xy',
                  scale_units='xy',
                  scale=1,
                  color=colors[i],
                  label=f'{var} (PC {i+1})')
        ax.text(eigvecs[0, i] * 1.1,
                eigvecs[1, i] * 1.1,
                var,  # Ajouter le nom de la variable ici.
                color=colors[i],
                fontsize=12,
                ha='center')

# Ajouter des labels et un titre.
ax.set_xlabel('Composante 1')
ax.set_ylabel('Composante 2')
ax.set_title('Cercle des corrélations avec statsmodels')

# Afficher la légende.
ax.legend()

# Afficher le graphique.
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

# Afficher les valeurs propres et les composantes principales.
print("Valeurs propres (explained variance) :")
print(eigvals)

print("\nVecteurs propres (composantes principales) :")
print(eigvecs)

Le code effectue les étapes suivantes :

- Calcule la matrice de corrélation des variables quantitatives.
- Effectue la décomposition en valeurs propres de cette matrice.
- Trie les valeurs propres et les vecteurs propres par ordre décroissant, afin d'identifier les directions principales de variance.
- Génère un cercle des corrélations sur un graphique pour visualiser ces vecteurs propres.

### Analyse des résultats

Le **cercle des corrélations** obtenu à partir de la décomposition en valeurs propres offre une vue visuelle de l’importance des relations entre les variables et de la direction de la plus grande variance dans les données. Dans ce graphique, les vecteurs propres représentent les directions dans lesquelles les variables contribuent le plus à la variance. Chaque vecteur est associé à une variable spécifique, et les directions des vecteurs indiquent la façon dont chaque variable est liée aux autres.

**Vecteurs propres (Composantes Principales) :**
- *Vecteurs propres* : Ce sont des directions principales (composantes principales) dans lesquelles les données varient le plus. Chaque vecteur propre est un axe dans un espace multidimensionnel. Plus l'angle entre un vecteur propre et une variable est petit, plus la variable est fortement corrélée avec cette direction. En d'autres termes, les variables qui pointent dans des directions similaires sont plus fortement corrélées entre elles.
- *Valeurs propres* : Elles représentent la variance expliquée par chaque composante principale. Plus une valeur propre est grande, plus la composante principale associée capture la variance des données. Si une composante a une valeur propre élevée, cela signifie qu'elle contient une proportion importante de l'information des données d'origine.

#### Interprétation des vecteurs dans le cercle des corrélations :

- Les vecteurs proches du bord du cercle (indiquant une direction forte de la variance) suggèrent que les variables sont fortement liées entre elles dans ces directions. Par exemple, si deux vecteurs sont proches dans le graphique, cela signifie que ces deux variables sont fortement corrélées.
- Les angles entre les vecteurs peuvent aussi donner des indices sur la corrélation entre les variables. Si deux vecteurs sont orientés dans des directions opposées (un angle de 180 degrés), les variables qu'ils représentent sont inversement corrélées. Si les vecteurs sont presque parallèles, les variables sont positivement corrélées.
- Si un vecteur est proche du centre du cercle, cela signifie que la variable associée à ce vecteur a une faible contribution à la variance expliquée et que sa relation avec les autres variables est moins forte. <br>

**Exemple**  :
Les vecteurs représentant *"flipper_length_mm"* et *"body_mass_g"* sont proches et pointent dans des directions similaires, cela indique que ces deux variables sont fortement corrélées. Ainsi, une variation dans la longueur du bec pourrait être associée à une variation similaire avec la masse corporelle. En revanche, le vecteur qui représente *"bill_length_mm"* est éloigné des autres vecteurs, cela suggère que cette variable contribue de manière unique à la variance des données, moins corrélée aux autres mesures morphologiques. De plus, en regardant le cercle des corrélations, on observe que les vecteurs de *"bill_length_mm"* et *"bill_depth_mm"* semblent être presque orthogonaux (c'est-à-dire perpendiculaires), ce qui suggère une faible ou aucune corrélation. En effet, dans la heatmap matrice de corrélation, on voit une valeur de -0.24, ce qui montre que ces caractéristiques ne sont pas corrélées.

En résumé, ce graphique de cercle des corrélations permet de comprendre rapidement quelles variables sont les plus liées et d'identifier les axes principaux de variance dans les données, tout en offrant une représentation visuelle claire des relations multidimensionnelles entre les différentes variables étudiées.

# Conclusion de la comparaison des bibliothèques pour réaliser des régressions linéaires et des ACP

- Au cours de ce tutoriel, nous avons comparé les bibliothèques **scikit-learn** et **statsmodels** lors de la réalisation de deux analyses statistiques principales :  
  - La régression linéaire  
  - L'Analyse en Composantes Principales (ACP)<br><br>   

- Il en ressort que les deux bibliothèques sont similaires et permettent de réaliser ces deux analyses de manière claire, efficace et comparable. Malgré des différences dans la préparation des données, les affichages et les méthodes de calcul, **les résultats finaux sont identiques**.  

- À noter et comme vu précédemment :  
  - **Statsmodels** est une bibliothèque dédiée aux statistiques "classiques", orientée vers la robustesse statistique et la vérification d’hypothèses.  
  - **Scikit-learn**, quant à elle, est davantage orientée vers le machine-learning (apprentissage automatique) et privilégie l’optimisation des performances.  

Ce faisant, les quelques différences notables proviennent majoritairement de ces différences :  
1. Pour la régression linéaire, **statsmodels** semblait plus robuste, mettant en avant des indicateurs statistiques détaillés, tandis que **scikit-learn** nécessitait un format de données particulier pour optimiser ses performances.  
2. Pour l’ACP, les deux bibliothèques donnaient des résultats identiques et des conclusions similaires, bien que leurs interfaces et options diffèrent légèrement.  

En conclusion, le choix entre ces bibliothèques dépend de vos besoins : **validation statistique rigoureuse** (statsmodels) ou **optimisation pour le machine-learning** (scikit-learn).
